<a href="https://colab.research.google.com/github/Gaurav7004/RSE_TOOL/blob/main/rse_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
# ! pip install pandarallel
from pandarallel import pandarallel
import json

In [ ]:
pandarallel.initialize()

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
# df = pd.read_csv('Data1.csv')

In [ ]:
## Data is the dictionary
data = {}

## Keys
key = ''

## values
values = []

## Open
with open('Example1.RSE', "r") as f:
    for line in f:
        line = line.strip()
        if line.startswith('#'):
            if key:
                key = key.split("'")[0]
                key = key.split("\t")[0]
                data[key] = [v for v in values if v and not v.startswith("'")]
            key = line.split("'")[0]
            key = key.split("\t")[0]
            values = []
        elif line.startswith("'"):
            pass
        else:
            line = line.split("'")[0]
            values.append(line.strip())

# add last group
if key:
    data[key] = [v for v in values if v and not v.startswith("'")]

print(json.dumps(data, indent=4))

{
    "#SAMPLING SUBSAMPLE2": [],
    "#GROUP 2": [
        "SEC   C sec",
        "ST_GR C NEWVAR"
    ],
    "#FILTER 1": [
        "AGE>=15"
    ],
    "#VARIABLE 3  ": [
        "POP\tN  NEWVAR",
        "LF\tN  NEWVAR",
        "WRK\tN  NEWVAR"
    ],
    "#FILE Data1.Xlsx": [],
    "#RENAME  5    ": [
        "STRMID C sec+st+strm+sstrm",
        "SS     C ss",
        "NSS    N  nss",
        "NSC    N  nsc",
        "MULT   N  mult"
    ],
    "#TRANSFORM 4  ": [
        "ST_GR=\"G1\"   in(ST,\"10,19,20,21\")",
        "ST_GR=\"G2\"   in(ST,\"11,12,13,14,15,16,17,18\")",
        "ST_GR=\"G3\"   Otherwise",
        "POP=1",
        "WRK=1  in(CWS,\"11,12,21,31,41,51,61,61,71,72\")",
        "WRK=0  NOT in(CWS,\"11,12,21,31,41,51,61,61,71,72\")",
        "LF=1      in(CWS,\"11,12,21,31,41,51,61,61,71,72,81\")",
        "LF=0      Otherwise"
    ],
    "#EST_RSE 7": [
        "S POP",
        "E POP",
        "R POP",
        "E 100*LF/POP",
        "R LF/POP",
        "E 100*WRK/

In [ ]:
# ## read the RSE file and extract the lines from it
# with open('Example1.RSE') as f:

#     ## Reading each line of the file 
#     lines = f.readlines()

# ## To generate keys of the dictionary 
# keys = []

# for line in lines:
#     # print(line)
#     line = line.strip()

#     if line.startswith("#"):

#         quote_index = line.find("'") 
#         if quote_index != -1:
#             key = line[1:quote_index].split()

#             # Check if the line is empty
#             if not line:
#                 # Empty line found, break out of the loop
#                 break

#             if key not in keys:
#                 keys.append(key)

# print(keys)

[['SAMPLING', 'SUBSAMPLE2'], ['GROUP', '2'], ['FILTER', '1'], ['VARIABLE', '3'], ['FILE', 'Data1.Xlsx'], ['RENAME', '5'], ['TRANSFORM', '4'], ['EST_RSE', '7']]


In [ ]:
### renaming the variables
### **********************

# rename the columns using a dictionary
new_columns = {
    'FSU': 'fsu',
    'NS_SSS': 'NSS'
}

df = df.rename(columns=new_columns)

df['STRMID'] = df['SEC'].apply(lambda x: f'0{x}' if x < 10 else str(x)) + \
                df['AST'].apply(lambda x: f'0{x}' if x < 10 else str(x)) + \
                 df[' STRM'].apply(lambda x: f'0{x}' if x < 10 else str(x)) + \
                  df['  SSTRM'].apply(lambda x: f'0{x}' if x < 10 else str(x))

df[' SSU'] = df[' FSU'].astype(str) + df[' IDNO'].apply(lambda x: f'0{x}' if x < 10 else str(x))

In [ ]:
df[' SSU']

In [ ]:
age = df['  AGE']

# Function to calculate the right age group
def get_age_group(age):
    if age < 5:
        return "01"
    elif age >= 5 and age < 10:
        return "02"
    elif age >= 10 and age < 20:
        return "03"
    elif age >= 20 and age < 60:
        return "04"
    elif age >= 60:
        return "05"
    else:
        return "00"

# Apply the get_age_group function to each row in the "age" column in parallel
df['AGE_GROUP'] = df['  AGE'].parallel_apply(get_age_group)


In [ ]:
### Age group with right age
def get_result(df):
    age = df['  AGE']
    ageGrp = df['AGE_GROUP']

    if ageGrp == "01" and age < 5:
        return "01"
    elif ageGrp == "02" and age >= 5 and age < 10:
        return "02"
    elif ageGrp == "03" and age >= 10 and age < 20:
        return "03"
    elif ageGrp == "04" and age >= 20 and age < 60:
        return "04"
    elif ageGrp == "05" and age>=60:
        return "00"
    else:
        return "00"

df['AGE_GROUP'] = df.parallel_apply(get_result, axis=1)

In [ ]:
def Y1_(df):
    df['newVar1'] = pd.Series([])
    df.loc[df['  GEDU_LVL'].isin([5, 6]), ('newVar1')] = df['    ERN_REG'] + df['   ERN_SELF']
    df.loc[~df['  GEDU_LVL'].isin([5, 6]), ('newVar1')] = 0

def Y2_(df):
    df['newVar2'] = pd.Series([])
    df.loc[df['  GEDU_LVL'].isin([7, 8, 9]), ('newVar2')] = df['    ERN_REG'] + df['   ERN_SELF']
    df.loc[~df['  GEDU_LVL'].isin([7, 8, 9]), ('newVar2')] = 0

def Y3_(df):
    df['newVar3'] = pd.Series([])
    df.loc[df['  GEDU_LVL'].isin([5, 6, 7, 8, 9]), ('newVar3')] = df['    ERN_REG'] + df['   ERN_SELF']
    df.loc[~df['  GEDU_LVL'].isin([5, 6, 7, 8, 9]), ('newVar3')] = 0

def Y4_(df):
    df['newVar4'] = pd.Series([])
    df.loc[df['   ACWS'].isin([11,12,21,31,41,51,61,61,71,72]), ('newVar4')] = 1
    df.loc[~df['   ACWS'].isin([11,12,21,31,41,51,61,61,71,72]), ('newVar4')] = 0

def Y5_(df):
    df['newVar5'] = pd.Series([])
    df.loc[df['   ACWS'].isin([11,12,21,31,41,51,61,61,71,72,81]), ('newVar5')] = 1
    df.loc[~df['   ACWS'].isin([11,12,21,31,41,51,61,61,71,72,81]), ('newVar5')] = 0

def Y6_(df):
    df['newVar6'] = 1

In [ ]:
Y1_(df), Y2_(df), Y3_(df), Y4_(df), Y5_(df), Y6_(df)

In [ ]:
# ### Example-2: Defining another variable

# print("select number of variables : ")

# n_var = int(input())

# ### creating no. of n_var
# for i in range(1, n_var+1):
#     # df['Y' + str(i)] = pd.Series([])
#     df['X' + str(i)] = pd.Series([])

#     if i == 1:
#         df.loc[df['  GEDU_LVL'].isin([5, 6]), ('X' + str(i))] = df['    ERN_REG'] + df['   ERN_SELF']
#         df.loc[~df['  GEDU_LVL'].isin([5, 6]), ('X' + str(i))] = 0
#     elif i == 2:
#         df.loc[df['  GEDU_LVL'].isin([7, 8, 9]), ('X' + str(i))] = df['    ERN_REG'] + df['   ERN_SELF']
#         df.loc[~df['  GEDU_LVL'].isin([5, 6]), ('X' + str(i))] = 0
#     elif i == 3:
#         df.loc[df['  GEDU_LVL'].isin([5, 6, 7, 8, 9]), ('X' + str(i))] = df['    ERN_REG'] + df['   ERN_SELF']
#         df.loc[~df['  GEDU_LVL'].isin([5, 6]), ('X' + str(i))] = 0
    

In [ ]:
df['newVar3'].unique()

In [ ]:
### Transform the variables



In [ ]:
df.to_csv('GROUP SEC ST AGE_Group.csv')

In [ ]:
df